In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# Any results you write to the current directory are saved as output.

/kaggle/input
/kaggle/input/isic2018
/kaggle/input/isic2018/isic2018_task1-2_validation_input
/kaggle/input/isic2018/isic2018_task1-2_validation_input/ISIC2018_Task1-2_Validation_Input
/kaggle/input/isic2018/isic2018_task1-2_training_input
/kaggle/input/isic2018/isic2018_task1-2_training_input/ISIC2018_Task1-2_Training_Input
/kaggle/input/isic2018/isic2018_task1_training_groundtruth
/kaggle/input/isic2018/isic2018_task1_training_groundtruth/ISIC2018_Task1_Training_GroundTruth
/kaggle/input/isic2018/ISIC2018_Task1_Training_GroundTruth
/kaggle/input/isic2018/ISIC2018_Task1_Training_GroundTruth/ISIC2018_Task1_Training_GroundTruth
/kaggle/input/isic2018/isic2018_task1-2_test_input
/kaggle/input/isic2018/isic2018_task1-2_test_input/ISIC2018_Task1-2_Test_Input
/kaggle/input/isic2018/ISIC2018_Task1-2_Training_Input
/kaggle/input/isic2018/ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input
/kaggle/input/isic2018/ISIC2018_Task1-2_Test_Input
/kaggle/input/isic2018/ISIC2018_Task1-2_Te

In [2]:
from __future__ import division
import scipy.io as sio
import scipy.misc as sc
from imageio import imread
import cv2
import glob

# Parameters
height = 256
width  = 256
channels = 3
Dataset_add = '/kaggle/input/isic2018/isic2018_task1-2_training_input'
Tr_add = 'ISIC2018_Task1-2_Training_Input'

Tr_list = glob.glob(Dataset_add+'/'+Tr_add+'/*.jpg')

# It contains 2594 training samples
Data_train_2018    = np.zeros([2594, height, width, channels])
Label_train_2018   = np.zeros([2594, height, width])

print('Reading ISIC 2018')
for idx in range(len(Tr_list)):
    if (idx+1)%50 == 0:
        print(idx+1)
    img = cv2.imread(Tr_list[idx])
    img = np.double(cv2.resize(img,(width,height),interpolation=cv2.INTER_LINEAR))
    Data_train_2018[idx, :,:,:] = img

    b = Tr_list[idx]    
    a = b[0:len(Dataset_add)]
    b = b[len(b)-16: len(b)-4] 
    add = ('/kaggle/input/isic2018/ISIC2018_Task1_Training_GroundTruth/ISIC2018_Task1_Training_GroundTruth/' + b +'_segmentation.png')    
    img2 = cv2.imread(add,0)
    img2 = cv2.resize(img2,(width,height),interpolation=cv2.INTER_LINEAR)
    Label_train_2018[idx, :,:] = img2    
         
print('Reading ISIC 2018 finished')

Reading ISIC 2018
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
Reading ISIC 2018 finished


In [3]:
Train_img      = Data_train_2018[0:1815,:,:,:]
Validation_img = Data_train_2018[1815:1815+259,:,:,:]
Test_img       = Data_train_2018[1815+259:2394,:,:,:]

Train_mask      = Label_train_2018[0:1815,:,:]
Validation_mask = Label_train_2018[1815:1815+259,:,:]
Test_mask       = Label_train_2018[1815+259:2394,:,:]

In [4]:
from __future__ import division
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils.vis_utils import plot_model as plot
from keras.optimizers import SGD
from keras.optimizers import *
from keras.layers import *  

def SegNet(nClasses=1, input_height=256, input_width=256):
    img_input = Input(shape=(input_height, input_width, 3))
    kernel_size = 3
    # encoder
    x = Conv2D(64, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # 128x128
    x = Conv2D(128, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # 64x64
    x = Conv2D(256, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # 32x32
    x = Conv2D(512, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # 16x16
    x = Conv2D(512, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # 8x8

    # decoder
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (kernel_size, kernel_size), padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = Conv2D(nClasses, (1, 1), padding='valid',
               kernel_initializer='he_normal')(x)

    x = Activation('sigmoid')(x)
    model = Model(img_input, x, name='SegNet')
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

Using TensorFlow backend.


In [5]:
from __future__ import division
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.callbacks import ModelCheckpoint, TensorBoard,ReduceLROnPlateau
from keras import callbacks
import pickle


# ===== normalize over the dataset 
def dataset_normalized(imgs):
    imgs_normalized = np.empty(imgs.shape)
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs-imgs_mean)/imgs_std
    for i in range(imgs.shape[0]):
        imgs_normalized[i] = ((imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i])-np.min(imgs_normalized[i])))*255
    return imgs_normalized

tr_data = Train_img
te_data = Test_img
val_data = Validation_img

tr_mask = Train_mask
te_mask = Test_mask
val_mask = Validation_mask

tr_mask    = np.expand_dims(tr_mask, axis=3)
te_mask    = np.expand_dims(te_mask, axis=3)
val_mask   = np.expand_dims(val_mask, axis=3)

print('ISIC18 Dataset loaded')
tr_data   = dataset_normalized(tr_data)
te_data   = dataset_normalized(te_data)
val_data  = dataset_normalized(val_data)

tr_mask   = tr_mask /255.
te_mask   = te_mask /255.
val_mask  = val_mask /255.

print('dataset Normalized')

ISIC18 Dataset loaded
dataset Normalized


In [6]:
model_segnet = SegNet()
model_segnet.summary()

print('Training')
batch_size = 8
nb_epoch = 30


mcp_save = ModelCheckpoint('weight_isic18_segnet', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
history = model_segnet.fit(tr_data,tr_mask,
              batch_size=batch_size,
              epochs=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(val_data, val_mask), callbacks=[mcp_save, reduce_lr_loss] )

model_segnet.save('segnet.h5')
  
print('Trained model saved')
with open('hist_isic18_segnet', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Model: "SegNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 128)     512  

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1815 samples, validate on 259 samples
Epoch 1/30
1815/1815 [==============================] - 53s 29ms/step - loss: 0.4029 - accuracy: 0.8612 - val_loss: 1.0627 - val_accuracy: 0.6240
Epoch 2/30
1815/1815 [==============================] - 44s 24ms/step - loss: 0.2922 - accuracy: 0.9072 - val_loss: 0.2924 - val_accuracy: 0.9082
Epoch 3/30
1815/1815 [==============================] - 44s 24ms/step - loss: 0.2489 - accuracy: 0.9202 - val_loss: 0.2787 - val_accuracy: 0.9192
Epoch 4/30
1815/1815 [==============================] - 44s 24ms/step - loss: 0.2254 - accuracy: 0.9246 - val_loss: 0.2586 - val_accuracy: 0.9127
Epoch 5/30
1815/1815 [==============================] - 44s 24ms/step - loss: 0.2071 - accuracy: 0.9301 - val_loss: 0.2209 - val_accuracy: 0.9211
Epoch 6/30
1815/1815 [==============================] - 44s 24ms/step - loss: 0.1871 - accuracy: 0.9351 - val_loss: 0.2216 - val_accuracy: 0.9256
Epoch 7/30
1815/1815 [==============================] - 44s 24ms/step - loss:

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score

te_data    = Test_img
te_mask = Test_mask
te_mask  = np.expand_dims(te_mask, axis=3)
te_data2  = dataset_normalized(te_data)

In [8]:
model_segnet = SegNet()
model_segnet.summary()
model_segnet.load_weights('weight_isic18_segnet')
predictions = model_segnet.predict(te_data, batch_size=8, verbose=1)
y_scores = predictions.reshape(predictions.shape[0]*predictions.shape[1]*predictions.shape[2]*predictions.shape[3], 1)
y_true = te_mask.reshape(te_mask.shape[0]*te_mask.shape[1]*te_mask.shape[2]*te_mask.shape[3], 1)
y_scores = np.where(y_scores>0.5, 1, 0)
y_true   = np.where(y_true>0.5, 1, 0)

threshold_confusion = 0.5
y_pred = np.empty((y_scores.shape[0]))
for i in range(y_scores.shape[0]):
    if y_scores[i]>=threshold_confusion:
        y_pred[i]=1
    else:
        y_pred[i]=0

Model: "SegNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
batch_normalization_11 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
activation_7 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
batch_normalization_12 (Batc (None, 128, 128, 128)     512  

In [9]:
F1_score = f1_score(y_true, y_pred, labels=None, average='binary', sample_weight=None)
print ("\nF1 score (F-measure): " +str(F1_score))


F1 score (F-measure): 0.8454459142483662


In [10]:
jaccard_index = jaccard_similarity_score(y_true, y_pred, normalize=True)
print ("\nJaccard similarity score: " +str(jaccard_index))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)



Jaccard similarity score: 0.940300703048706
